In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

accounts = pd.read_csv("ravenstack_dataset/accounts.csv")
churn_events = pd.read_csv("ravenstack_dataset/churn_events.csv")
feature_usage = pd.read_csv("ravenstack_dataset/feature_usage.csv")
subscriptions = pd.read_csv("ravenstack_dataset/subscriptions.csv")
support_tickets = pd.read_csv("ravenstack_dataset/support_tickets.csv")

accounts.shape, churn_events.shape,feature_usage.shape,subscriptions.shape,support_tickets.shape

churn_events.info()

support_tickets.head()

churn_events["churn_date"] = pd.to_datetime(churn_events["churn_date"],format="mixed")

accounts["signup_date"] = pd.to_datetime(accounts["signup_date"],format="mixed")

accounts.info()

feature_usage["usage_date"] = pd.to_datetime(feature_usage["usage_date"],format="mixed")

feature_usage.info()

date_columns = ["start_date","end_date"]
subscriptions[date_columns] = subscriptions[date_columns].apply(lambda x: pd.to_datetime(x,format="mixed"))

subscriptions.info()

date_columns = ["submitted_at","closed_at"]
support_tickets[date_columns] = support_tickets[date_columns].apply(pd.to_datetime,format="mixed",dayfirst=True)

support_tickets["closed_at"].dtype

churn_events["account_id"].nunique()

churn_events.sort_values("account_id").head(10)

churn_events.duplicated().sum()

churn_clean = (churn_events.sort_values("churn_date").drop_duplicates("account_id",keep="first"))

accounts["Churned"] = accounts["account_id"].isin(churn_events["account_id"]).astype(int) 

accounts["Churned"].value_counts()

accounts["Churned"].mean()

accounts.groupby("industry")["Churned"].mean()

accounts.groupby("country")["Churned"].mean().sort_values(ascending=False)

accounts.groupby("plan_tier")["Churned"].mean().sort_values(ascending=False)

df = accounts.merge(subscriptions,on="account_id",how="left")

df = df.merge(churn_clean[["account_id","churn_date"]],on="account_id",how="left")

df["Churned"].value_counts(normalize=True) 

df.groupby("plan_tier_x")["Churned"].mean()

df.groupby("plan_tier_y")["Churned"].mean()

df.columns

usage_summary = (feature_usage.groupby("subscription_id").agg(avg_usage_count=("usage_count","mean"),active_days=("usage_date","nunique")).reset_index())

df = df.merge(usage_summary,on="subscription_id",how="left")

df.groupby("Churned")[["avg_usage_count","active_days"]].mean()

plt.figure(figsize=(10,8))
sns.boxplot(data=df,x="Churned",y="avg_usage_count")
plt.title("Churn vs Feature_usage")
plt.show()

ticket_summary = (support_tickets.groupby("account_id").agg(ticket_count=("ticket_id","count"),avg_resolution=("resolution_time_hours","mean")).reset_index())

df = df.merge(ticket_summary,on="account_id",how="left")

df[["ticket_count","avg_resolution"]] = df[["ticket_count","avg_resolution"]].fillna(0)

df.groupby("Churned")[["ticket_count","avg_resolution"]].mean()

df.groupby(["upgrade_flag","downgrade_flag"])["Churned"].mean()

df.to_csv("churn_analysis_py.csv",index=False)

